First set the working directory to scripts in order to import the different functions

In [1]:
import os
os.chdir("scripts")

In [ ]:
from evaluation import evaluate
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset
from find_alpha import linear_search_alpha

If you want to do query rewriting you would need an open ai api key, otherwise run this cell as is without filling it. \
Also if you want to generate train and test set as we generated it you need this key, but we advise to download our already generated train and test set from [here](https://epflch-my.sharepoint.com/:f:/g/personal/marco_giuliano_epfl_ch/EomT1E_2ZaFCkB7zpx-jDAMBxLz6UP8NGtdBNvK10RaYHQ?e=w8s5QY).

In [3]:
openai_api_key = "sk-"

If you want to use reranking you will need a cohere api key

In [4]:
cohere_api_key = "COHERE_API_KEY"

# Choose the model

In this section you can choose between the different model architectures we explained in the report. \
To use BM25 set "BM25" \
To use BGE set "BGE" \
To use finetuned BGE set retriever_model to "BGE_Finetuned" \
To use hybrid search set "Hybrid_BGE" and "Hybrid_Finetuned"

In [5]:
retriever_model = "BM25"
query_rewriting = True
reranking = False
top_k = 10
alpha = 0.9

# To evaluate the different models run this cell

Evaluation of the model

In [6]:
test_data = EmbeddingQAFinetuneDataset.from_json("../test_dataset_new.json")

In [ ]:
eval= evaluate(test_data,
    retriever_model,
    top_k=top_k,
    verbose=True,
    query_rewriting=query_rewriting,
    reranking=reranking,
    alpha=alpha,
    top_k_before_reranking= 50,
    openai_api_key= openai_api_key,
    cohere_api_key= cohere_api_key,
    )

In [ ]:
print(eval)

Linear search for alpha in case of hybrid search:
Find best alpha parameter, the retriever model should be either "Hybrid_BGE" or "Hybrid_Finetuned"

In [8]:
val_data = EmbeddingQAFinetuneDataset.from_json("../val_dataset_new.json")

In [ ]:
alphas = [0.1, 0.2, 0.3, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65 , 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
alphas_score = linear_search_alpha(val_data, retriever_model, alphas)

In [6]:
best_alpha = max(alphas_score, key = alphas_score.get)
print(best_alpha)

# Generating train and test questions

In [ ]:
from create_train_val_test import create_train_val_test
from datasets import load_dataset

ds = load_dataset("ashraq/financial-news-articles")

With this function you can recreate the train and test dataset that we have, but you would need an Openai api key

To create train set, set split = "train" \
To create val set, set split = "val" \
To create test set, set split = "test"

In [2]:
split = "test"

In [3]:
openai_api_key = openai_api_key #IF you have not set the openai api key in the previous cell, you can set it here as "sk-".

In [ ]:
create_train_val_test(ds, split, openai_api_key)

# Finetune BGE

Here you can finetune yourself the BGE model

In [2]:
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset
from finetuning import finetune_BGE

train_dataset = EmbeddingQAFinetuneDataset.from_json("../train_dataset_new.json")
val_dataset = EmbeddingQAFinetuneDataset.from_json("../val_dataset_new.json")

In [ ]:
finetune_BGE(train_dataset, val_dataset, "../models/BGE_Finetuned")